# **Introduction**

This is where you can find the workflow of how we trained out Cell Identifer Model.

It begins with looking for data online, which is saved in Structures.

We then use Cellpose-SAM[1] to segment individual cells within these images into .png files, which are saved in Cells.

## **Workflow**

### Requirements

We begin by installing all required libraries from requirements.txt, which can be seen in the main file directory. In order to speed up image processing, CUDA is installed, which allows us to make use of the GPU to greatly decreases run time. Ensure you have the latest version of python installed.

In [1]:
pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


If not already created, this code will create all necessary folders. Please run this for the other cells to work!

In [2]:
import os

def create_folder(foldername):
    try:
        os.mkdir(foldername)
    except FileExistsError:
        print(f"{foldername} already exists.")

folders = ["./Data/Structures", "./Data/Train", "./Data/Validate"]

for folder in folders:
    create_folder(folder)

./Data/Structures already exists.
./Data/Train already exists.
./Data/Validate already exists.


### Image Processing

Import Libraries

In [3]:
import cellpose
from cellpose import io, models
import numpy as np
from skimage import color, util, measure
from tqdm.notebook import tqdm
import os



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.5 
platform:       	win32 
python version: 	3.13.5 
torch version:  	2.7.1+cu118! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 




Initialise Cellpose model (Cellpose-SAM) to create masks for our images. Change the value of "gpu=True" to False if you do not have a Nivida GPU in your computer.

In [ ]:
model = models.CellposeModel(gpu=True)

Image processing function. In order to shorten processing time, maxcells is set to 2000. This is assuming each image file provided will at least have 1 cell in the picture, thus producing 2000 cell images at the minimum to be fed to our machine learning model. It can be set higher for other purposes. See code for more in-depth explanation.

In [ ]:
def generatecellcrops(folderpath, cellname):
    newfolderpath = f"./Data/Train/{cellname}_Cell"

    try:
        os.mkdir(newfolderpath)

        filenames = os.listdir(folderpath)
        maxcells = 2000
        pbar = tqdm(total=maxcells, desc=f"Processing {cellname}")

        cellnumber = 1
        for file in filenames:
            if cellnumber > maxcells:
                break

            imagepath = os.path.join(folderpath, file)
            image = io.imread_2D(imagepath) # read image file
            image_gray = color.rgb2gray(image) # gray scale image for easier masking
            inp = (image_gray * 255).astype(np.uint8) # convert from floating-point to 8-bit image
            
            masks, flows, styles = model.eval(inp, invert=True) # mask image, invert is set to True assuming the image data provided is brightfield microscopy

            for prop in measure.regionprops(masks, intensity_image=image_gray): # get properties of mask  
                if prop.area < 50 or prop.mean_intensity < 0.1: # remove mask noise pickup from poorly prepared cell samples
                    continue

                label = prop.label
                minr, minc, maxr, maxc = prop.bbox # top-left and bottom-right of bounding box

                cell_mask = (masks == label) # True when pixels are in current region

                cell_image = np.zeros_like(image) # copy the shape of the original image
                
                if image.ndim == 2: # if image is gray scale
                    cell_image[cell_mask] = image[cell_mask] # copy over the pixels that are inside the region

                else: # if image has color
                    for c in range(image.shape[2]):
                        channel = image[...,c]
                        channel_out = np.zeros_like(channel)
                        channel_out[cell_mask] = channel[cell_mask]
                        cell_image[...,c] = channel_out

                cell_crop = cell_image[minr:maxr, minc:maxc] # crop to bounding box

                if cell_crop.dtype != np.uint8: # convert to 8-bit image to save as .png
                    cell_crop = util.img_as_ubyte(cell_crop)

                if not np.any(cell_crop):
                    continue

                cellimagepath = os.path.join(newfolderpath, f"{cellname}_Cell_{cellnumber:04d}.png")
                io.imsave(cellimagepath, cell_crop)
                cellnumber += 1
                pbar.update(1)
            
        pbar.close()
        print(f"Done! Extracted {cellnumber} cells.")
    except FileExistsError:
        print(f"{newfolderpath} already exists. Skipping.")
        return


Access the image files in your folder, which should be placed under Dataset/Structures.

In [ ]:
directory = "./Data/Structures"

try:
    folders = os.listdir(directory)
except FileNotFoundError:
    print("No cell image folders found.")

for folder in folders:
    folderpath = os.path.join(directory, folder)
    generatecellcrops(folderpath, folder)

## Training our Model

Our model is finetuned from EfficientNet-LiteB0[2] from timm using PyTorch.

Import Libraries.

In [2]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import timm
from tqdm import trange, tqdm

Create folders if they do not already exist and split data. 80% of data will be used as training data and 20% as validation data. 

In [3]:
import shutil

folders = ["./Data", "./Data/Train", "./Data/Validate"]

for folder in folders:
    create_folder(folder)

directory = "./Data/Train"
try:
    folders = os.listdir(directory) # access all classifications
except FileNotFoundError:
    print("No data folders found.")

for folder in folders:
    try:
        folderpath = os.path.join(directory, folder)
        files = os.listdir(folderpath) # access all images

        validatepath = os.path.join("./Data/Validate", folder)
        create_folder(validatepath) # create folders in Data Validate

        size = len(files)
        validate = (size * 2) // 10

        for file in files[:validate]: # move the 20% to validate
            path = os.path.join(directory, folder, file)
            newpath = os.path.join(validatepath, file)
            shutil.move(path, newpath)

    except FileNotFoundError:
        continue # skip to the next folder

./Data already exists.
./Data/Train already exists.
./Data/Validate already exists.


Training cycle.

In [4]:
def train_epoch(loader, model, criterion, optimizer, device):
    model.train()
    total_loss, total_correct = 0.0, 0

    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * imgs.size(0)
        total_correct += (outputs.argmax(1) == labels).sum().item()

    avg_loss = total_loss / len(loader.dataset)
    avg_acc = total_correct / len(loader.dataset)
    return avg_loss, avg_acc

Training cycle evaluation.

In [5]:
def eval_epoch(loader, model, criterion, device):
    model.eval()
    total_loss, total_correct = 0.0, 0

    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss    = criterion(outputs, labels)

            total_loss    += loss.item() * imgs.size(0)
            total_correct += (outputs.argmax(1) == labels).sum().item()

    avg_loss = total_loss / len(loader.dataset)
    avg_acc  = total_correct / len(loader.dataset)
    return avg_loss, avg_acc

The main runner code block to finetune the model. Currently we are using 20 epochs to cut down on processing time, given access to more computing resources, the number of epochs can be increased.

Similarly to Image Processing, finetuning will also use the GPU if it is available, it is highly reccomended to use a GPU.

Please run the other cells under "Training our model" before running this block!

In [6]:
num_epochs = 50 # change number of epochs here

# ImageNet normalization statistics
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Process images to tensors for training
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

train_ds = datasets.ImageFolder('Data/Train', transform=train_transform) # Training on cell crop images
val_ds = datasets.ImageFolder('Data/Validate', transform=val_transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=4)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=4)

num_classes = len(train_ds.classes) # number of classifications

if torch.cuda.is_available():
    print("GPU and CUDA detected, using GPU.\n")
else:
    print("GPU and/or CUDA not detected, using CPU.\n")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Loading in EfficientNet-LiteB0
model = timm.create_model('efficientnet_lite0', pretrained=True)

# Replace the classifier head
in_features = model.classifier.in_features
model.classifier = nn.Linear(in_features, num_classes)

# Load model in CPU/GPU
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

best_val_acc = 0.0
patience = 10
no_improve_epochs = 0
pbar = tqdm(total=num_epochs, desc="Training") # Progress bar to estimate training time

for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(train_loader, model, criterion, optimizer, device)

    val_loss, val_acc = eval_epoch(val_loader, model, criterion, device)
    
    scheduler.step()

    pbar.set_postfix({
        "T_Loss": f"{train_loss:.4f}",
        "T_Acc":  f"{train_acc:.4f}",
        "V_Loss": f"{val_loss:.4f}",
        "V_Acc":  f"{val_acc:.4f}"
    })

    # save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        no_improve_epochs = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        no_improve_epochs += 1

    if no_improve_epochs >= patience:
        print(f"Early stopping at epoch {epoch+1}. No improvement in the last {patience} epochs.")
        break
    
    pbar.update(1)

pbar.close()

try:
    os.remove("Classifications.txt")
except FileNotFoundError:
    print("Classifications.txt not found, creating new file.")
    
with open("Classifications.txt", "w") as file: # write classifications file
    file.write(train_ds.classes[0])
    for Class in train_ds.classes[1:]:
        file.write(f"\n{Class}")

GPU and CUDA detected, using GPU.



Training:  38%|███▊      | 19/50 [15:54<25:56, 50.22s/it, T_Loss=0.1751, T_Acc=0.9489, V_Loss=0.2466, V_Acc=0.9286]

Early stopping at epoch 20. No improvement in the last 10 epochs.
Classifications.txt not found, creating new file.


Test the finetuned model. Cell images will first be cropped and saved to ./Output/Cells before identification takes place.

In [4]:
import torch
import timm
import torch.nn as nn
from PIL import Image
from torchvision import transforms
import cellpose
from cellpose import models, io
from skimage import color, util, measure
import numpy as np

outputdirectory = "./Output"
cellcroppath = "./Output/Cells"
create_folder(outputdirectory)
create_folder(cellcroppath)

cellpose_model = models.CellposeModel(gpu=True)

Classes = []
with open("Classifications.txt", "r") as file:
    Classes = [line.strip() for line in file]
num_classes = len(Classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = timm.create_model('efficientnet_lite0', pretrained=False)
in_features = model.classifier.in_features
model.classifier = nn.Linear(in_features, num_classes)

checkpoint = torch.load('best_model.pth', map_location=device)
model.load_state_dict(checkpoint)

model = model.to(device)
model.eval()

mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

# test image
img_path = 'test.jpg'
img = io.imread(img_path) # read image file
if img.ndim == 2:
    img_gray = img
else:
    img_gray = color.rgb2gray(img) # gray scale image for easier masking
inp = (img_gray * 255).astype(np.uint8) # convert from floating-point to 8-bit image

masks, flows, styles = cellpose_model.eval(inp) # mask image

props = measure.regionprops(masks, intensity_image=img_gray) # get cell mask properties

for cellnumber, prop in enumerate(props, 1):
    if prop.area < 50 or prop.mean_intensity < 0.1: # remove mask noise pickup from poorly prepared cell samples
        continue

    label = prop.label
    minr, minc, maxr, maxc = prop.bbox # top-left and bottom-right of bounding box

    cell_mask = (masks == label) # True when pixels are in current region

    cell_img = np.zeros_like(img) # copy the shape of the original image

    if img.ndim == 2: # if image is gray scale
        cell_img[cell_mask] = img[cell_mask] # copy over the pixels that are inside the region

    else: # if image has color
        for c in range(img.shape[2]):
            channel = img[...,c]
            channel_out = np.zeros_like(channel)
            channel_out[cell_mask] = channel[cell_mask]
            cell_img[...,c] = channel_out

    cell_crop = cell_img[minr:maxr, minc:maxc] # crop to bounding box

    if cell_crop.dtype != np.uint8: # convert to 8-bit image to save as .png
        cell_crop = util.img_as_ubyte(cell_crop)

    individualcellcroppath = os.path.join(cellcroppath, f"cell_{cellnumber}.png")
    io.imsave(individualcellcroppath, cell_crop)

    cell_crop = Image.fromarray(cell_crop)
        
    input_tensor = val_transform(cell_crop).unsqueeze(0).to(device)

    with torch.no_grad():
        logits = model(input_tensor)
        probs  = torch.softmax(logits, dim=1)
        pred_class = logits.argmax(dim=1).item()
        pred_conf  = probs[0, pred_class].item()

    print(f"Predicted class index of cell {cellnumber}: {Classes[pred_class]}  (confidence: {pred_conf:.2%})")


./Output already exists.
./Output/Cells already exists.
Predicted class index of cell 1: Animal_Cell_Mitosis_Cell  (confidence: 63.78%)
Predicted class index of cell 2: Motorneurons_Cell  (confidence: 36.88%)
Predicted class index of cell 3: Ovary_Of_Lilium_Cell  (confidence: 53.40%)
Predicted class index of cell 4: Stomach_Wall_Cell  (confidence: 62.33%)
Predicted class index of cell 5: Onion_Root_Tip_Cell  (confidence: 31.35%)
Predicted class index of cell 6: Stomach_Wall_Cell  (confidence: 28.89%)
Predicted class index of cell 7: Stomach_Wall_Cell  (confidence: 19.98%)
Predicted class index of cell 8: Motorneurons_Cell  (confidence: 58.18%)
Predicted class index of cell 9: Stomach_Wall_Cell  (confidence: 47.29%)
Predicted class index of cell 10: Stomach_Wall_Cell  (confidence: 24.73%)
Predicted class index of cell 11: Stomach_Wall_Cell  (confidence: 65.20%)
Predicted class index of cell 12: Ovary_Of_Lilium_Cell  (confidence: 36.71%)
Predicted class index of cell 13: Onion_Root_Tip_C

# **Citations**

[1] M. Pachitariu, M. Rariden, and C. Stringer, “Cellpose-SAM: superhuman generalization for cellular segmentation,” May 01, 2025, bioRxiv. doi: 10.1101/2025.04.28.651001.

[2] M. Tan and Q. V. Le, “EfficientNet: Rethinking model scaling for convolutional neural networks,” arXiv preprint arXiv:1905.11946, 2020.